# Assignment 2

#### First Name: Lorenzo Andrés
#### Last Name: Vigo del Rosso


## 1. Load Data from JSON

In [1]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Twitter Analysis")\
.getOrCreate()

In [17]:
df_twitter = spark.read.json("corona_tweet_new.json")

In [18]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [19]:
### From the user nestec col select the following cols only id_str,followers_count,friends_count and created at 
# (2 points)
from pyspark.sql.functions import col 
df_twitter=df_twitter.select(col("created_at"),
                             col("favorite_count"),
                             col("hashtags"),
                             col("id"),
                             col("in_reply_to_status_id"),
                             col("in_reply_to_user_id_str"),
                             col("location"),
                             col("reply_count"),
                             col("retweet_count"),
                             col("source"),
                             col("user.id_str").alias("user_id_str"),
                             col("user.followers_count").alias("user_followers_count"),
                             col("user.friends_count").alias("user_friends_count"),
                             col("user.created_at").alias("user_created_at")
                            )

In [20]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [21]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter.count()

15894

In [22]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regualr expresion and the group would be 1 for this regular expression.
#(4 points)
from pyspark.sql.functions import regexp_extract, col

df_twitter=df_twitter.withColumn("extracted_source", regexp_extract('source',r'<a [^>]+>([^<]+)', 1)) # Use what's given in the commment  
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter Web Client|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
+----------

In [23]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter.rdd.map(tuple)

In [24]:
# Create a temporay table in memory with name as twitter (1 point)
df_twitter.createTempView("TWITTER")

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

In [147]:
# Using RDD
rdd_twitter.map(lambda tweet: (tweet[10], 1)).reduceByKey(lambda a, b: a + b).count()

14094

In [148]:
# Using DataFrame
df_twitter.select("user_id_str").distinct().count()

14094

In [149]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(DISTINCT user_id_str) FROM TWITTER').show()

+---------------------------+
|count(DISTINCT user_id_str)|
+---------------------------+
|                      14094|
+---------------------------+



#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

In [150]:
# Using RDD
rdd_twitter.map(lambda tweet: (tweet[10], 1)).reduceByKey(lambda a, b: a + b).filter(lambda user: user[1] > 1).count()

1016

In [151]:
# Using DataFrame
df_twitter.groupby("user_id_str").count().filter("count > 1").count()

1016

In [153]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(*) FROM (SELECT COUNT(user_id_str) AS tweet_count FROM TWITTER GROUP BY user_id_str) WHERE tweet_count > 1').show()

+--------+
|count(1)|
+--------+
|    1016|
+--------+



#### 2.3 Get total number unique extracted_source (1 point each)

In [154]:
# Using RDD
rdd_twitter.map(lambda tweet: (tweet[14], 1)).reduceByKey(lambda a, b: a + b).count()

133

In [155]:
# Using DataFrame
df_twitter.select("extracted_source").distinct().count()

133

In [156]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(DISTINCT extracted_source) FROM TWITTER').show()

+--------------------------------+
|count(DISTINCT extracted_source)|
+--------------------------------+
|                             133|
+--------------------------------+



#### 2.4 Get top 5 most used extracted_source

In [157]:
# Using RDD (5 points)
rdd_twitter.map(lambda tweet: (tweet[14], 1)).reduceByKey(lambda a, b: a + b).takeOrdered(5, lambda s: -1 * s[1])

[('Twitter for Android', 6262),
 ('Twitter for iPhone', 5698),
 ('Twitter Web App', 2878),
 ('Twitter for iPad', 428),
 ('Twitter Web Client', 136)]

In [158]:
# Using DataFrame (2 points)
df_twitter.groupby("extracted_source").count().sort("count", ascending = False).show(5)

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+
only showing top 5 rows



In [159]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('SELECT extracted_source, COUNT(extracted_source) as count FROM TWITTER GROUP BY extracted_source ORDER BY count DESC LIMIT 5').show()

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+



#### 2.5 Get count of distinct hastags used ( 5 point each) 

In [160]:
# Using RDD
rdd_twitter.flatMap(lambda tweet: tweet[2]).map(lambda hashtags: (hashtags, 1)).reduceByKey(lambda a, b: a + b).count()

1215

In [161]:
# Using DataFrame
import pyspark.sql.functions as functions
df_twitter.select(functions.explode(col('hashtags'))).distinct().count()

1215

In [162]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(*) FROM (SELECT DISTINCT EXPLODE(hashtags) FROM TWITTER)').show()

+--------+
|count(1)|
+--------+
|    1215|
+--------+



#### 2.6 Get top 5 hashtags

In [163]:
# Using RDD (4 points)
rdd_twitter.flatMap(lambda tweet: tweet[2]).map(lambda hashtags: (hashtags, 1)).reduceByKey(lambda a, b: a + b).takeOrdered(5, lambda s: -1 * s[1])

[('طبق_القدرات_للثانويه_ياريس', 385),
 ('Corona', 319),
 ('OilPrice', 251),
 ('COVID19', 125),
 ('corona', 123)]

In [164]:
# Using DataFrame (2 points)
df_twitter.select(functions.explode(col('hashtags'))).groupby("col").count().sort("count", ascending = False).show(5)

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+
only showing top 5 rows



In [166]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('SELECT col, COUNT(col) as count FROM (SELECT EXPLODE(hashtags) FROM TWITTER) GROUP BY col ORDER BY count DESC LIMIT 5').show()

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+



#### 2.7 Get total number of tweets which are retweeted more than 100 times

In [167]:
# Using RDD
# TODO review: rdd_twitter.filter(lambda tweet: tweet[8] > 100).map(lambda tweet: (tweet[3], 1)).reduceByKey(lambda a, b: a + b).count() gives different value
rdd_twitter.filter(lambda tweet: tweet[8] > 100).count()

15753

In [168]:
# Using DataFrame
df_twitter.filter("retweet_count > 100").count()

15753

In [169]:
# Using spark.sql and the temporay table.
# TODO review: distinct gives different result
spark.sql('SELECT COUNT(id) FROM TWITTER WHERE retweet_count > 100').show()

+---------+
|count(id)|
+---------+
|    15753|
+---------+



#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [170]:
# Using RDD
from operator import itemgetter

rdd_twitter.map(lambda tweet: (tweet[6], [tweet[3], tweet[8]])).groupByKey().mapValues(lambda x: sorted(list(x), key=itemgetter(1), reverse=True)[:3]).collect()

[('India',
  [['1252332114948874240', 9988],
   ['1252252336921206787', 9976],
   ['1252254519116746754', 9973]]),
 ('Pakistan',
  [['1252334264248606720', 9988],
   ['1252251912084357121', 9975],
   ['1252252126694309888', 9973]]),
 ('USA',
  [['1252331777806524416', 9994],
   ['1252254239805579264', 9987],
   ['1252335464750735362', 9982]]),
 ('Italy',
  [['1252252106750377996', 9994],
   ['1252251206027816960', 9984],
   ['1252330500670337024', 9971]]),
 ('Canada',
  [['1252335430323888128', 9997],
   ['1252254877939531776', 9992],
   ['1252252082825986051', 9987]]),
 ('China',
  [['1252335780707684352', 9998],
   ['1252253596516843520', 9993],
   ['1252255562525560832', 9984]]),
 ('Chile',
  [['1252253612140490759', 9988],
   ['1252334891951427585', 9984],
   ['1252253710182481920', 9978]]),
 ('UK',
  [['1252333018578145280', 9991],
   ['1252252091822870529', 9989],
   ['1252254043973603329', 9985]]),
 ('Mexico',
  [['1252253843145912320', 9998],
   ['1252255209776189442', 9994],
 

In [171]:
# Using DataFrame
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

df_twitter.select('id','location','retweet_count', rank().over(Window().partitionBy("location").orderBy(col("retweet_count").desc())).alias('rank')).sort("retweet_count",
    ascending=False).filter('rank < 4').sort("location", "rank").show()

+-------------------+--------+-------------+----+
|                 id|location|retweet_count|rank|
+-------------------+--------+-------------+----+
|1252335430323888128|  Canada|         9997|   1|
|1252254877939531776|  Canada|         9992|   2|
|1252252082825986051|  Canada|         9987|   3|
|1252253612140490759|   Chile|         9988|   1|
|1252334891951427585|   Chile|         9984|   2|
|1252253710182481920|   Chile|         9978|   3|
|1252335780707684352|   China|         9998|   1|
|1252253596516843520|   China|         9993|   2|
|1252255562525560832|   China|         9984|   3|
|1252334028092399622| Germany|         9999|   1|
|1252330902325248000| Germany|         9997|   2|
|1252252295510855682| Germany|         9990|   3|
|1252332114948874240|   India|         9988|   1|
|1252252336921206787|   India|         9976|   2|
|1252254519116746754|   India|         9973|   3|
|1252252106750377996|   Italy|         9994|   1|
|1252251206027816960|   Italy|         9984|   2|


In [172]:
# Using spark.sql and the temporay table.
spark.sql('''SELECT id, location, retweet_count FROM (SELECT *, RANK() OVER (PARTITION BY location ORDER BY retweet_count DESC) as rank FROM TWITTER) WHERE rank < 4 ORDER BY location, rank ASC''').show()

+-------------------+--------+-------------+
|                 id|location|retweet_count|
+-------------------+--------+-------------+
|1252335430323888128|  Canada|         9997|
|1252254877939531776|  Canada|         9992|
|1252252082825986051|  Canada|         9987|
|1252253612140490759|   Chile|         9988|
|1252334891951427585|   Chile|         9984|
|1252253710182481920|   Chile|         9978|
|1252335780707684352|   China|         9998|
|1252253596516843520|   China|         9993|
|1252255562525560832|   China|         9984|
|1252334028092399622| Germany|         9999|
|1252330902325248000| Germany|         9997|
|1252252295510855682| Germany|         9990|
|1252332114948874240|   India|         9988|
|1252252336921206787|   India|         9976|
|1252254519116746754|   India|         9973|
|1252252106750377996|   Italy|         9994|
|1252251206027816960|   Italy|         9984|
|1252330500670337024|   Italy|         9971|
|1252253843145912320|  Mexico|         9998|
|125225520

#### 2.9 Total number of tweets per country

In [173]:
# Using RDD (3 points)
rdd_twitter.map(lambda tweet: (tweet[6], 1)).reduceByKey(lambda a, b: a + b).collect()

[('India', 1480),
 ('Pakistan', 1470),
 ('USA', 1539),
 ('Italy', 1422),
 ('Canada', 1441),
 ('China', 1457),
 ('Chile', 1410),
 ('UK', 1376),
 ('Mexico', 1409),
 ('Spain', 1464),
 ('Germany', 1426)]

In [174]:
# Using DataFrame (2 points)
df_twitter.groupBy("location").count().show()

+--------+-----+
|location|count|
+--------+-----+
| Germany| 1426|
|   India| 1480|
|   China| 1457|
|   Chile| 1410|
|   Italy| 1422|
|   Spain| 1464|
|     USA| 1539|
|  Mexico| 1409|
|      UK| 1376|
|  Canada| 1441|
|Pakistan| 1470|
+--------+-----+



In [175]:
# Using spark.sql and the temporay table. (1 point)
spark.sql('SELECT location, COUNT(id) FROM TWITTER GROUP BY location').show()

+--------+---------+
|location|count(id)|
+--------+---------+
| Germany|     1426|
|   India|     1480|
|   China|     1457|
|   Chile|     1410|
|   Italy|     1422|
|   Spain|     1464|
|     USA|     1539|
|  Mexico|     1409|
|      UK|     1376|
|  Canada|     1441|
|Pakistan|     1470|
+--------+---------+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [177]:
# Using DataFrame
df_twitter.withColumn("hashtags", col("hashtags").cast("string")).write.partitionBy('location').csv("location", header = 'true')

#### 3.2 Save the data as parquet files (1 points)

In [145]:
# Using DataFrame
df_twitter.write.partitionBy('location').parquet("location.parquet")